In [5]:
import json
from collections import defaultdict

In [6]:
def rename_metric(metric):
    if metric == 'F-measure':
        return 'f1'
    if metric == 'Accuracy':
        return 'acc'
    if metric == 'Runtime':
        return 'time'
    raise ValueError('Unknown metric')

In [7]:
def get_results(metric):
    Res = {metric: {'Carcinogenesis': {'CELOE': '', 'ELTL': '', 'ECII': '', 'EvoLearner': '', 'NCES$_{\\text{LSTM}}$': '', 'NCES$_{\\text{GRU}}$': '',\
                                       'NCES$_{\\text{ST}}$': '', 'NCES$_{\\text{ST+GRU}}$': '', 'NCES$_{\\text{ST+LSTM}}$': '',\
                                       'NCES$_{\\text{GRU+LSTM}}$': '', 'NCES$_{\\text{ST+GRU+LSTM}}$': ''},
                   'Mutagenesis': {'CELOE': '', 'ELTL': '', 'ECII': '', 'EvoLearner': '', 'NCES$_{\\text{LSTM}}$': '', 'NCES$_{\\text{GRU}}$': '',\
                                       'NCES$_{\\text{ST}}$': '', 'NCES$_{\\text{ST+GRU}}$': '', 'NCES$_{\\text{ST+LSTM}}$': '',\
                                       'NCES$_{\\text{GRU+LSTM}}$': '', 'NCES$_{\\text{ST+GRU+LSTM}}$': ''},
                    'Semantic Bible': {'CELOE': '', 'ELTL': '', 'ECII': '', 'EvoLearner': '', 'NCES$_{\\text{LSTM}}$': '', 'NCES$_{\\text{GRU}}$': '',\
                                       'NCES$_{\\text{ST}}$': '', 'NCES$_{\\text{ST+GRU}}$': '', 'NCES$_{\\text{ST+LSTM}}$': '',\
                                       'NCES$_{\\text{GRU+LSTM}}$': '', 'NCES$_{\\text{ST+GRU+LSTM}}$': ''},
                    'Vicodi': {'CELOE': '', 'ELTL': '', 'ECII': '', 'EvoLearner': '', 'NCES$_{\\text{LSTM}}$': '', 'NCES$_{\\text{GRU}}$': '',\
                                       'NCES$_{\\text{ST}}$': '', 'NCES$_{\\text{ST+GRU}}$': '', 'NCES$_{\\text{ST+LSTM}}$': '',\
                                       'NCES$_{\\text{GRU+LSTM}}$': '', 'NCES$_{\\text{ST+GRU+LSTM}}$': ''}
                   }}
    for kb in ['carcinogenesis', 'mutagenesis', 'semantic_bible', 'vicodi']:
        for model in ['celoe', 'eltl', 'ecii', 'evolearner', 'NCES', 'NCES_Ensemble']:
            if model not in ['NCES', 'NCES_Ensemble']:
                with open(f'datasets/{kb}/Results/concept_learning_avg_results__{model}.json') as file:
                    results = json.load(file)
                rename_model = model.upper() if model != 'evolearner' else 'EvoLearner'
                if model == 'ecii':
                    if metric != 'Accuracy':
                        Res[metric][' '.join(word.capitalize() for word in kb.split('_'))][rename_model] =\
                    str(round(100*results[metric]['mean'],2))+'$\pm$'+str(round(100*results[metric]['std'],2))
                    else:
                        Res[metric][' '.join(word.capitalize() for word in kb.split('_'))][rename_model] =\
                    '-'+'$\pm$'+'-'
                else:
                    Res[metric][' '.join(word.capitalize() for word in kb.split('_'))][rename_model] =\
                    str(round(results[metric]['mean'],2))+'$\pm$'+str(round(results[metric]['std'],2))
            elif model == 'NCES':
                with open(f'datasets/{kb}/Results/{model}.json') as file:
                    results = json.load(file)
                new_metric = rename_metric(metric)
                for m in results: 
                    rename_model = 'NCES$_'+'{\\text{'+m+'}}$' if m != 'SetTransformer' else 'NCES$_'+'{\\text{'+'ST'+'}}$'
                    Res[metric][' '.join(word.capitalize() for word in kb.split('_'))][rename_model] =\
                    str(round(results[m][new_metric]['mean'][0],2))+'$\pm$'+str(round(results[m][new_metric]['std'][0],2))
            elif model == 'NCES_Ensemble':
                with open(f'datasets/{kb}/Results/{model}.json') as file:
                    results = json.load(file)
                new_metric = rename_metric(metric)
                for m in results:
                    rename_model = 'NCES$_'+'{\\text{'+m+'}}$' if not 'SetTransformer' in m \
                    else 'NCES$_'+'{\\text{'+m.replace('SetTransformer', 'ST')+'}}$'
                    Res[metric][' '.join(word.capitalize() for word in kb.split('_'))][rename_model] =\
                    str(round(results[m][new_metric]['mean'][0],2))+'$\pm$'+str(round(results[m][new_metric]['std'][0],2))
    return Res

In [26]:
def get_results_shuffle(metric):
    Res = {metric: {'Carcinogenesis': {'NCES$_{\\text{LSTM}}$': '', 'NCES$_{\\text{GRU}}$': '',\
                                       'NCES$_{\\text{ST}}$': ''},
                   'Mutagenesis': {'NCES$_{\\text{LSTM}}$': '', 'NCES$_{\\text{GRU}}$': '',\
                                       'NCES$_{\\text{ST}}$': ''},
                    'Semantic Bible': {'NCES$_{\\text{LSTM}}$': '', 'NCES$_{\\text{GRU}}$': '',\
                                       'NCES$_{\\text{ST}}$': ''},
                    'Vicodi': {'NCES$_{\\text{LSTM}}$': '', 'NCES$_{\\text{GRU}}$': '',\
                                       'NCES$_{\\text{ST}}$': ''}
                   }}
    for kb in ['carcinogenesis', 'mutagenesis', 'semantic_bible', 'vicodi']:
        with open(f'datasets/{kb}/Results/NCES_Shuffle.json') as file:
            results = json.load(file)
        with open(f'datasets/{kb}/Results/NCES.json') as file:
            results_initial = json.load(file)
        new_metric = rename_metric(metric)
        difference_results = {m: {new_metric: results[m][new_metric]['mean'][0]-results_initial[m][new_metric]['mean'][0]} for m in results}
        
        for m in difference_results: 
            rename_model = 'NCES$_'+'{\\text{'+m+'}}$' if m != 'SetTransformer' else 'NCES$_'+'{\\text{'+'ST'+'}}$'
            Res[metric][' '.join(word.capitalize() for word in kb.split('_'))][rename_model] =\
            str(round(difference_results[m][new_metric],2))
    return Res

In [27]:
Res1 = get_results_shuffle('F-measure')

In [28]:
Res1

{'F-measure': {'Carcinogenesis': {'NCES$_{\\text{LSTM}}$': '-2.14',
   'NCES$_{\\text{GRU}}$': '2.07',
   'NCES$_{\\text{ST}}$': '0.0'},
  'Mutagenesis': {'NCES$_{\\text{LSTM}}$': '1.96',
   'NCES$_{\\text{GRU}}$': '19.06',
   'NCES$_{\\text{ST}}$': '0.0'},
  'Semantic Bible': {'NCES$_{\\text{LSTM}}$': '-10.74',
   'NCES$_{\\text{GRU}}$': '-9.78',
   'NCES$_{\\text{ST}}$': '0.0'},
  'Vicodi': {'NCES$_{\\text{LSTM}}$': '-0.49',
   'NCES$_{\\text{GRU}}$': '2.71',
   'NCES$_{\\text{ST}}$': '0.0'}}}

In [29]:
def write_results(metric):
    data = get_results(metric)
    with open(f'{metric}.txt', 'w') as file:
        for key in data:
            file.write("\\begin{tabular}{@{}lccccccc@{}}\n")
            file.write("\t\t\\toprule\n")
            file.write("\t\t& \\multicolumn{4}{c}{"+"$"+metric+"(\%)$}\\\\\n")
            file.write("\t\t&"+" & ".join(["\\textbf{"+kb+"}" for kb in data[key]])+"\\\\\n")
            file.write("\\midrule\n")
            for model in data[key]['Carcinogenesis']:
                file.write("\t\t"+model+" & "+" & ".join([data[key][kb][model] for kb in data[key]])+"\\\\\n")

In [30]:
def write_results_shuffle(metric):
    data = get_results_shuffle(metric)
    with open(f'{metric}_shuffle.txt', 'w') as file:
        for key in data:
            file.write("\\begin{tabular}{@{}lccccccc@{}}\n")
            file.write("\t\t\\toprule\n")
            file.write("\t\t& \\multicolumn{4}{c}{"+"$"+metric+"(\%)$}\\\\\n")
            file.write("\t\t&"+" & ".join(["\\textbf{"+kb+"}" for kb in data[key]])+"\\\\\n")
            file.write("\\midrule\n")
            for model in data[key]['Carcinogenesis']:
                file.write("\t\t"+model+" & "+" & ".join([data[key][kb][model] for kb in data[key]])+"\\\\\n")

In [31]:
write_results_shuffle('F-measure')

In [32]:
write_results_shuffle('Accuracy')